In [1]:
import os
import selenium
from selenium import webdriver
import time
import pandas as pd
import numpy as np 
# Listemos cada uno de los pdb que vamos a analizar
submission_dir = 'brood_pdb'
dir_list = np.sort(list(os.listdir(submission_dir)))


#Abramos  mediante el driver el servidor de PERMM
driver = webdriver.Chrome()
driver.get('https://permm.phar.umich.edu/permm_server_cgopm')

In [2]:
binding_l = []
blm_l     = []
bbb_po_l  = []
caco2_po_l= []
pampa_po_l= []
plasma_m_l= []
energy_l  = []
name = []
k= 0
for q in dir_list:
    try:
        k+=1
        print(k,q)
        # Escogemos el boton de pdb
        choose_file = driver.find_element("name", "pdbFile")
        # Ponemos la ruta que necesitamos 
        file_location = os.path.join("/home/chema-chiles/Downloads/ligandos_analisis/chromedriver_linux64/", "brood_pdb", q)
        # Subimos el archivo pdb de la molecula al boton
        choose_file.send_keys(file_location)


        #Cambiemos la temperatura
        temp_box = driver.find_element("name",'temperature')
        temp_box.clear()
        temp_box.send_keys('310')

        # Le decimos que nos calcule todos los parametros
        driver.find_element("xpath","/html/body/div[1]/div/div[3]/div[2]/div/form/ul/li[5]/input[2]").click()

        # Pongamos mi correo
        # email_box = driver.find_element("name",'userEmail')
        # email_box.send_keys('jose.maria.montes.montiel@gmail.com')

        # Mandar la tarea
        driver.find_element("name",'submit').click()

        #Entremos al tab que tiene los resultados 
        time.sleep(25)
        driver.find_element("link text","Results").click()
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(25)

        #Guardemos los resultados de lo computado
        binding = driver.find_element("xpath","/html/body/table[2]/tbody/tr[1]").text[30:-9]
        blm     = driver.find_element("xpath","/html/body/table[2]/tbody/tr[2]").text[26:]
        bbb_po  = driver.find_element("xpath","/html/body/table[2]/tbody/tr[3]").text[30:]
        caco2_po= driver.find_element("xpath","/html/body/table[2]/tbody/tr[4]").text[32:]
        pampa_po= driver.find_element("xpath","/html/body/table[2]/tbody/tr[5]").text[35:]
        plasma_m= driver.find_element("xpath","/html/body/table[3]/tbody/tr[1]/td").text.split("\n")[-1][35:]
        energy  = driver.find_element("xpath","/html/body/table[3]/tbody/tr[1]/td").text.split("\n")[9:-6]

        energy_local = []
        for i in [i.split("; ") for i in energy]:
            energy_local.append(float(i[1]))

        #Agreguemos a la lista 
        binding_l.append(binding)
        blm_l.append(blm)
        bbb_po_l.append(bbb_po)
        caco2_po_l.append(caco2_po)
        pampa_po_l.append(pampa_po)
        plasma_m_l.append(plasma_m)
        energy_l.append(energy_local)
        name.append(q)

        #Preparemos cosas para el nuevo ciclo
        driver.close()
        time.sleep(5)
        driver.switch_to.window(driver.window_handles[0])
        driver.find_element("xpath","/html/body/div[1]/div/div[3]/div[2]/div/ul/li[4]/button").click()
    except Exception as e:
        print(f"Error al procesar el archivo {q}:")
        binding_l.append(np.nan)
        blm_l.append(np.nan)
        bbb_po_l.append(np.nan)
        caco2_po_l.append(np.nan)
        pampa_po_l.append(np.nan)
        plasma_m_l.append(np.nan)
        energy_l.append(np.nan)
        name.append(q)
        
        #Preparemos cosas para el nuevo ciclo
        driver.close()
        time.sleep(5)
        driver.switch_to.window(driver.window_handles[0])
        driver.find_element("xpath","/html/body/div[1]/div/div[3]/div[2]/div/ul/li[4]/button").click()
        continue        
driver.close()

In [3]:
logp = {
    "name": name,
    "Free energy of binding (DOPC) kcal/mol":binding_l,
    "Log of perm. coeff. - BLM":blm_l,
    "Log of perm. coeff - BBB (Po)":bbb_po_l,
    "Log of perm. coeff - CACO2 (Po)":caco2_po_l,
    "Log of perm. coeff - PAMPA-DS (Po)":pampa_po_l,
    "Log of perm. coeff - plasma membr":plasma_m_l,
}

energias = {
    "name": name,
    "energia": energy_l
}

In [4]:
logp = pd.DataFrame(logp)
energias =  pd.DataFrame(energias)
logp.to_csv("logp_expermmderiv.csv",index=False)
energias.to_csv("energias_ex_permmderiv.csv",index=False)

In [5]:
logp